In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy

In [82]:
data = pd.read_csv('data/cgm_to_meal.csv')
data = data.dropna()
cgm_diff = data['cgm'].diff()
data['diff'] = cgm_diff
data = data.dropna()
data = data.reset_index(drop=True)

In [56]:
# non_meal_diff = data.loc[(data['meal'] == 0) & (data['diff'] < 20) & (data['diff'] > -20)]['diff']
# meal_diff = data.loc[(data['meal'] == 1) & (data['diff'] < 20) & (data['diff'] > -20) ]['diff']
# plt.hist([meal_diff, non_meal_diff], bins = 50, label=['meal', 'non-meal'])
# plt.legend()

In [89]:
near_param = 5

meal_indices = data.index[data['meal'] == 1]
meal_data_X = []
for idx in meal_indices:
    near_idx_points = data.iloc[list(range(idx - 1, idx - near_param - 1, -1))]
    meal_data_X.append(near_idx_points['diff'].to_list())

non_meal_indices = data.index[data['meal'] == 0]
non_meal_data_X = []
for idx in non_meal_indices:
    near_idx_points = data.iloc[list(range(idx - 1, idx - near_param - 1, -1))]
    non_meal_data_X.append(near_idx_points['diff'].to_list())

In [96]:
meal_data_Y = np.ones(len(meal_data_X))
non_meal_data_Y = np.ones(len(non_meal_data_X))
meal_data_X = np.array(meal_data_X)
non_meal_data_X = np.array(non_meal_data_X)